# * Interim : 2025 Target Revenue TRUE
    96 Areas

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2025, 6, 2)

## ETL Process...

### Step 1 : Import Data Source

In [42]:
''' Rawdata '''

# Target_Revenue_TRUE_Y2025
src_file = '../../data/interim/Revenue/Target_Revenue_TRUE_Y2025.xlsx'
src_df_cols = ['CUST_TYPE', 'TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME', 'METRIC_VALUE', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_CD', 'AREA_DESC']

src_b2c_sheet = 'B2C Rawdata'
src_b2c_df = pd.read_excel(src_file, sheet_name=src_b2c_sheet, index_col=None) 
src_b2c_df = src_b2c_df.loc[src_b2c_df['AREA_TYPE']=="HH"]
src_b2c_df['CUST_TYPE'] = 'B2C'
src_b2c_df = src_b2c_df[src_df_cols]

src_b2b_sheet = 'B2B Rawdata'
src_b2b_df = pd.read_excel(src_file, sheet_name=src_b2b_sheet, index_col=None) 
src_b2b_df['CUST_TYPE'] = 'B2B'
src_b2b_df = src_b2b_df[src_df_cols]

src_df = pd.concat([src_b2c_df, src_b2b_df])
src_df.rename(columns={'AREA_CD': 'AREA_KEY', 'METRIC_VALUE': 'TARGET_MTH'}, inplace=True)
src_df['COMP_CD'] = 'TRUE' #src_df['COMP_CD'].astype(str)
# src_df = src_df.replace(np.nan, None)
src_df = src_df.reset_index(drop=True)

print(f'\nsrc_df : {src_df.shape[0]} rows, {src_df.shape[1]} columns')
src_df#.tail(3)


src_df : 2310 rows, 10 columns


,CUST_TYPE,TM_KEY_MTH,METRIC_CD,METRIC_NAME,TARGET_MTH,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC
0,B2C,202501,TB1R000100,Prepaid Revenue : TMH,5.710683e+06,TRUE,T,HH,902033,ANG THONG
1,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.081782e+07,TRUE,T,HH,902034,CHAI NAT
2,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.663773e+07,TRUE,T,HH,902035,KANCHANABURI
3,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.987740e+07,TRUE,T,HH,902036,LOP BURI
4,B2C,202501,TB1R000100,Prepaid Revenue : TMH,3.576186e+07,TRUE,T,HH,902037,NAKHON PATHOM
...,...,...,...,...,...,...,...,...,...,...
2305,B2B,202502,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide
2306,B2B,202503,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide
2307,B2B,202504,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide
2308,B2B,202505,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide


In [43]:
''' Add Columns '''

def product_group(v_cd):
    cd = v_cd
    result = ''
    if re.search('B1', cd): result = 'Prepaid'
    elif re.search('B2', cd): result = 'Postpaid'
    elif re.search('B3', cd): result = 'TOL'
    elif re.search('B4', cd): result = 'TVS'
    else: result = 'Unknown' 
    return result

src_df['PRODUCT_GRP'] = src_df.apply(lambda x: product_group(x['METRIC_CD']), axis=1)
# src_df['FREQUENCY'] = np.where(src_df['PRODUCT_GRP']=='Prepaid', 'DAY', 'BILL')
src_df['FREQUENCY'] = np.where(src_df['PRODUCT_GRP']=='Prepaid', 'Daily', 'Bill Cycle')
src_df['TM_KEY_YR'] = src_df['TM_KEY_MTH'].apply(str).str[:4].astype(int)

src_df#.tail(3)

,CUST_TYPE,TM_KEY_MTH,METRIC_CD,METRIC_NAME,TARGET_MTH,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,PRODUCT_GRP,FREQUENCY,TM_KEY_YR
0,B2C,202501,TB1R000100,Prepaid Revenue : TMH,5.710683e+06,TRUE,T,HH,902033,ANG THONG,Prepaid,Daily,2025
1,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.081782e+07,TRUE,T,HH,902034,CHAI NAT,Prepaid,Daily,2025
2,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.663773e+07,TRUE,T,HH,902035,KANCHANABURI,Prepaid,Daily,2025
3,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.987740e+07,TRUE,T,HH,902036,LOP BURI,Prepaid,Daily,2025
4,B2C,202501,TB1R000100,Prepaid Revenue : TMH,3.576186e+07,TRUE,T,HH,902037,NAKHON PATHOM,Prepaid,Daily,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,B2B,202502,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide,Postpaid,Bill Cycle,2025
2306,B2B,202503,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide,Postpaid,Bill Cycle,2025
2307,B2B,202504,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide,Postpaid,Bill Cycle,2025
2308,B2B,202505,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide,Postpaid,Bill Cycle,2025


In [44]:
''' Master Data '''

# DIM_TIME
dt_file = '../CFW/data/dim_time.csv'
# dt_cols = ['TM_KEY_YR', 'MONTH_SHORT', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH']
dt_cols = ['TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAY_NO', 'DAYS_IN_MONTH', 'PERIODFLAG']
dt_df = pd.read_csv(dt_file, usecols=dt_cols)

# DIM_MOOC_AREA
mooc_file = '../CFW/data/dim_mooc_area.csv'
mooc_cols = ['ZONE_TYPE', 'TEAM_CODE', 'ORGID_G', 'TDS_SGMD', 'ORGID_R', 'TDS_RGM_CODE', 'ORGID_H', 'HOP_HINT', 'TDS_PROVINCE', 'PROVINCE_ENG', 'PROVINCE_TH', 'ORGID_HH', 'D_CLUSTER', 'CCAATT', 'REMARK']
mooc_df = pd.read_csv(mooc_file, usecols=mooc_cols)
mooc_df = mooc_df.loc[(mooc_df['REMARK']!='Dummy') & (mooc_df['TEAM_CODE']!='ไม่ระบุ') & (mooc_df['HOP_HINT']!='True Corp')]

# Create HH level
mooc_hh_df = mooc_df[['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_H', 'HOP_HINT', 'ORGID_HH', 'D_CLUSTER']].drop_duplicates()
mooc_hh_df.dropna(how='all', inplace=True)
mooc_hh_df['AREA_KEY'] = mooc_hh_df['ORGID_HH'].astype(int).astype(str)

In [45]:
''' Portion Data '''

portion_file = '../CFW/data/revenue_portion.xlsx'

new_existing_sheet = 'New & Existing'
new_existing_df = pd.read_excel(portion_file, sheet_name=new_existing_sheet, index_col=None) 
new_existing_df = new_existing_df.loc[new_existing_df['TM_KEY_YR']==2025]
new_existing_cols = ['PRODUCT_GRP', 'TM_KEY_MTH', 'NEW', 'EXIST']
new_existing_df = new_existing_df[new_existing_cols]
# new_existing_df

bill_cycle_sheet = 'Bill Cycle'
bill_cycle_df = pd.read_excel(portion_file, sheet_name=bill_cycle_sheet, index_col=None) 
bill_cycle_df = bill_cycle_df.loc[bill_cycle_df['TM_KEY_YR']==2025]
bill_cycle_cols = ['METRIC_CD', 'TM_KEY_YR', 'BILLING_DAY', 'BILL_PORTION']
bill_cycle_df = bill_cycle_df[bill_cycle_cols]
# bill_cycle_df

# src_df_cols = ['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME', 'METRIC_VALUE', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_CD', 'AREA_DESC']
# src_df = src_df[src_df_cols]
# src_df.rename(columns={'AREA_CD': 'AREA_KEY', 'METRIC_VALUE': 'MTH_VALUE'}, inplace=True)

### Step 2 : Aggregate Data

In [7]:
# ''' Example DataFrame '''

# src_df.tail(3)
# dt_df.tail(3)
# mooc_df.tail(3)
# mooc_h_df.tail(3)
# mooc_h_df.loc[mooc_h_df['ORGID_H'].str.contains('^0')].tail(3)

In [46]:
''' Filter Rawdata

    TB1R000100	Prepaid Revenue : TMH
    TB2R010100	Postpaid Revenue B2C : TMH
    TB2R020100	Postpaid Revenue B2B : TMH
    TB3R000100	TOL Revenue
    TB4R000100	TVS Revenue
'''

''' Filter '''
raw_df = src_df.copy()
raw_df = raw_df.loc[raw_df['TM_KEY_MTH']>=202505]
# raw_df = raw_df.loc[raw_df['METRIC_CD']=='TB1R000100'] #Prepaid Revenue : TMH
# raw_df = raw_df.loc[raw_df['TM_KEY_MTH'].isin([202501, 202502, 202503])]
# raw_df = raw_df.loc[raw_df['METRIC_CD'].isin(['TB1R000100', 'TB2R010100', 'TB3R000100', 'TB4R000100'])]

''' Data Test '''
# raw_df = raw_df.loc[raw_df['TM_KEY_MTH']==202504]
# raw_df = raw_df.loc[raw_df['METRIC_CD']=='TB1R000100']
# raw_df = raw_df.loc[raw_df['AREA_KEY']=='902033']
# raw_df = raw_df.loc[raw_df['AREA_KEY'].isna()]

raw_df = raw_df.reset_index(drop=True)
print(f'\nraw_df : {raw_df.shape[0]} rows, {raw_df.shape[1]} columns')
raw_df#.tail(3)


raw_df : 770 rows, 13 columns


,CUST_TYPE,TM_KEY_MTH,METRIC_CD,METRIC_NAME,TARGET_MTH,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,PRODUCT_GRP,FREQUENCY,TM_KEY_YR
0,B2C,202505,TB1R000100,Prepaid Revenue : TMH,2.936362e+07,TRUE,T,HH,907030,"BKK : Bang Khen, Lat Phrao, Wang Thonglang",Prepaid,Daily,2025
1,B2C,202505,TB1R000100,Prepaid Revenue : TMH,2.296415e+07,TRUE,T,HH,907016,"BKK : Bang Sue, Chatuchak",Prepaid,Daily,2025
2,B2C,202505,TB1R000100,Prepaid Revenue : TMH,2.729176e+07,TRUE,T,HH,907017,"BKK : Don Mueang, Sai Mai, Lak Si",Prepaid,Daily,2025
3,B2C,202505,TB1R000100,Prepaid Revenue : TMH,3.999441e+07,TRUE,T,HH,907019,"BKK : Lat Krabang, Nong Chok, Khlong Sam Wa",Prepaid,Daily,2025
4,B2C,202505,TB1R000100,Prepaid Revenue : TMH,3.128469e+07,TRUE,T,HH,907020,"BKK : Min Buri, Khan Na Yao, Bueng Kum",Prepaid,Daily,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,B2C,202506,TB4R000100,TVS Revenue,5.789441e+06,TRUE,T,HH,906110,SURAT THANI,TVS,Bill Cycle,2025
766,B2C,202506,TB4R000100,TVS Revenue,8.447717e+05,TRUE,T,HH,906111,TRANG,TVS,Bill Cycle,2025
767,B2C,202506,TB4R000100,TVS Revenue,3.849959e+05,TRUE,T,HH,906112,YALA,TVS,Bill Cycle,2025
768,B2B,202505,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide,Postpaid,Bill Cycle,2025


In [47]:
''' Join New & Existing '''

merge_new_existing_df = pd.merge(raw_df, new_existing_df, how='left', on=['PRODUCT_GRP', 'TM_KEY_MTH'])
merge_new_existing_df['TARGET_MTH_NEW'] = merge_new_existing_df['TARGET_MTH'] * merge_new_existing_df['NEW']
merge_new_existing_df['TARGET_MTH_EXIST'] = merge_new_existing_df['TARGET_MTH'] * merge_new_existing_df['EXIST']
merge_new_existing_df = merge_new_existing_df[['CUST_TYPE', 'TM_KEY_YR', 'TM_KEY_MTH', 'PRODUCT_GRP', 'FREQUENCY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_KEY', 'AREA_DESC', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST']]
merge_new_existing_df.tail(3)

,CUST_TYPE,TM_KEY_YR,TM_KEY_MTH,PRODUCT_GRP,FREQUENCY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST
767,B2C,2025,202506,TVS,Bill Cycle,TB4R000100,TVS Revenue,TRUE,T,HH,906112,YALA,3.849959e+05,1.776661e+03,3.832193e+05
768,B2B,2025,202505,Postpaid,Bill Cycle,TB2R020100,Postpaid Revenue B2B : TMH,TRUE,T,P,P,Nationwide,3.182800e+08,2.379846e+07,2.941419e+08
769,B2B,2025,202506,Postpaid,Bill Cycle,TB2R020100,Postpaid Revenue B2B : TMH,TRUE,T,P,P,Nationwide,3.182800e+08,2.955784e+07,2.883169e+08


In [48]:
''' Allocate Daily '''

day_df = merge_new_existing_df.loc[merge_new_existing_df['FREQUENCY']=='Daily']

day_df = pd.merge(day_df, dt_df, how='left', on='TM_KEY_MTH')

day_df['TARGET_DAY'] = day_df['TARGET_MTH'] / day_df['DAYS_IN_MONTH']
day_df['TARGET_DAY_NEW'] = day_df['TARGET_MTH_NEW'] / day_df['DAYS_IN_MONTH']
day_df['TARGET_DAY_EXIST'] = day_df['TARGET_MTH_EXIST'] / day_df['DAYS_IN_MONTH']

day_df = day_df[['CUST_TYPE', 'TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH', 'PERIODFLAG', 'PRODUCT_GRP', 'FREQUENCY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_KEY', 'AREA_DESC', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST', 'TARGET_DAY', 'TARGET_DAY_NEW', 'TARGET_DAY_EXIST']]
day_df.tail(3)

,CUST_TYPE,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,DAYS_IN_MONTH,PERIODFLAG,PRODUCT_GRP,FREQUENCY,METRIC_CD,...,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST,TARGET_DAY,TARGET_DAY_NEW,TARGET_DAY_EXIST
5853,B2C,2025,202506,2025026,20250628,30,N,Prepaid,Daily,TB1R000100,...,T,HH,906112,YALA,8.159928e+06,1.654762e+06,6.505166e+06,271997.611956,55158.745926,216838.866031
5854,B2C,2025,202506,2025026,20250629,30,N,Prepaid,Daily,TB1R000100,...,T,HH,906112,YALA,8.159928e+06,1.654762e+06,6.505166e+06,271997.611956,55158.745926,216838.866031
5855,B2C,2025,202506,2025027,20250630,30,EMQ,Prepaid,Daily,TB1R000100,...,T,HH,906112,YALA,8.159928e+06,1.654762e+06,6.505166e+06,271997.611956,55158.745926,216838.866031


In [49]:
''' Allocate Bill Cycle '''

bill_df = merge_new_existing_df.loc[merge_new_existing_df['FREQUENCY']=='Bill Cycle']
bill_df = pd.merge(bill_df, bill_cycle_df, how='left', on=['METRIC_CD', 'TM_KEY_YR'])

day_in_month_df = dt_df[['TM_KEY_MTH', 'DAYS_IN_MONTH']].drop_duplicates()
bill_df = pd.merge(bill_df, day_in_month_df, how='left', on='TM_KEY_MTH')
bill_df['BILLING_DAY'] = np.where(bill_df['BILLING_DAY']==1, bill_df['DAYS_IN_MONTH'], bill_df['BILLING_DAY'])

period_flag_df = dt_df[['TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAY_NO', 'PERIODFLAG']].drop_duplicates()
period_flag_df.rename(columns={'DAY_NO': 'BILLING_DAY'}, inplace=True)
bill_df = pd.merge(bill_df, period_flag_df, how='left', on=['TM_KEY_MTH', 'BILLING_DAY'])

bill_df['TARGET_DAY'] = bill_df['TARGET_MTH'] * bill_df['BILL_PORTION']
bill_df['TARGET_DAY_NEW'] = bill_df['TARGET_MTH_NEW'] * bill_df['BILL_PORTION']
bill_df['TARGET_DAY_EXIST'] = bill_df['TARGET_MTH_EXIST'] * bill_df['BILL_PORTION']

bill_df = bill_df[['CUST_TYPE', 'TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH', 'PERIODFLAG', 'PRODUCT_GRP', 'FREQUENCY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_KEY', 'AREA_DESC', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST', 'TARGET_DAY', 'TARGET_DAY_NEW', 'TARGET_DAY_EXIST']]

''' Test '''
# bill_df = bill_df.loc[bill_df['PRODUCT_GRP']=='TOL']

bill_df.tail(3)

,CUST_TYPE,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,DAYS_IN_MONTH,PERIODFLAG,PRODUCT_GRP,FREQUENCY,METRIC_CD,...,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST,TARGET_DAY,TARGET_DAY_NEW,TARGET_DAY_EXIST
5777,B2B,2025,202506,2025025,20250622,30,N,Postpaid,Bill Cycle,TB2R020100,...,T,P,P,Nationwide,318280000.0,2.955784e+07,2.883169e+08,1.563996e+07,1.452443e+06,1.416760e+07
5778,B2B,2025,202506,2025026,20250625,30,N,Postpaid,Bill Cycle,TB2R020100,...,T,P,P,Nationwide,318280000.0,2.955784e+07,2.883169e+08,3.578867e+07,3.323602e+06,3.241950e+07
5779,B2B,2025,202506,2025026,20250628,30,N,Postpaid,Bill Cycle,TB2R020100,...,T,P,P,Nationwide,318280000.0,2.955784e+07,2.883169e+08,4.426796e+07,4.111051e+06,4.010053e+07


In [50]:
''' Concat Day & Bill '''

day_and_bill_df = pd.concat([day_df, bill_df])

''' Test '''
# day_and_bill_df = day_and_bill_df.loc[day_and_bill_df['PRODUCT_GRP']=='Postpaid']
# day_and_bill_df = day_and_bill_df.loc[day_and_bill_df['AREA_KEY'].isin(['P', '902033'])]

print(f'day_and_bill_df : {day_and_bill_df.shape[0]} rows, {day_and_bill_df.shape[1]} columns')
day_and_bill_df.tail(3)

day_and_bill_df : 11636 rows, 22 columns


,CUST_TYPE,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,DAYS_IN_MONTH,PERIODFLAG,PRODUCT_GRP,FREQUENCY,METRIC_CD,...,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST,TARGET_DAY,TARGET_DAY_NEW,TARGET_DAY_EXIST
5777,B2B,2025,202506,2025025,20250622,30,N,Postpaid,Bill Cycle,TB2R020100,...,T,P,P,Nationwide,318280000.0,2.955784e+07,2.883169e+08,1.563996e+07,1.452443e+06,1.416760e+07
5778,B2B,2025,202506,2025026,20250625,30,N,Postpaid,Bill Cycle,TB2R020100,...,T,P,P,Nationwide,318280000.0,2.955784e+07,2.883169e+08,3.578867e+07,3.323602e+06,3.241950e+07
5779,B2B,2025,202506,2025026,20250628,30,N,Postpaid,Bill Cycle,TB2R020100,...,T,P,P,Nationwide,318280000.0,2.955784e+07,2.883169e+08,4.426796e+07,4.111051e+06,4.010053e+07


In [51]:
''' Join Area '''

merge_hh_df = pd.merge(day_and_bill_df, mooc_hh_df, how='left', on='AREA_KEY')
# merge_hh_df = pd.merge(day_and_bill_df.loc[day_and_bill_df['AREA_TYPE']=='HH'], mooc_hh_df, how='left', on='AREA_KEY')

''' Test '''
# merge_hh_df = merge_hh_df.loc[merge_hh_df['PRODUCT_GRP']=='TOL']
# merge_hh_df = merge_hh_df.loc[merge_hh_df['AREA_KEY'].isin(['P', '902033'])]
# merge_hh_df = merge_hh_df.loc[merge_hh_df['TM_KEY_DAY']==20250302]

merge_hh_df = merge_hh_df[['CUST_TYPE', 'TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH', 'PERIODFLAG', 'PRODUCT_GRP', 'FREQUENCY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_KEY', 'AREA_DESC', 'ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_H', 'HOP_HINT', 'ORGID_HH', 'D_CLUSTER', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST', 'TARGET_DAY', 'TARGET_DAY_NEW', 'TARGET_DAY_EXIST']]
print(f'merge_hh_df : {merge_hh_df.shape[0]} rows, {merge_hh_df.shape[1]} columns')
merge_hh_df.tail(3)

merge_hh_df : 11636 rows, 29 columns


,CUST_TYPE,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,DAYS_IN_MONTH,PERIODFLAG,PRODUCT_GRP,FREQUENCY,METRIC_CD,...,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST,TARGET_DAY,TARGET_DAY_NEW,TARGET_DAY_EXIST
11633,B2B,2025,202506,2025025,20250622,30,N,Postpaid,Bill Cycle,TB2R020100,...,NaN,NaN,NaN,NaN,318280000.0,2.955784e+07,2.883169e+08,1.563996e+07,1.452443e+06,1.416760e+07
11634,B2B,2025,202506,2025026,20250625,30,N,Postpaid,Bill Cycle,TB2R020100,...,NaN,NaN,NaN,NaN,318280000.0,2.955784e+07,2.883169e+08,3.578867e+07,3.323602e+06,3.241950e+07
11635,B2B,2025,202506,2025026,20250628,30,N,Postpaid,Bill Cycle,TB2R020100,...,NaN,NaN,NaN,NaN,318280000.0,2.955784e+07,2.883169e+08,4.426796e+07,4.111051e+06,4.010053e+07


In [52]:
''' Aggregate P, G, H, HH '''

# agg_cols = ['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'DAY_VALUE', 'MTH_VALUE'] # , 'FREQUENCY', 'REMARK'
agg_cols = ['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'FREQUENCY', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST', 'TARGET_DAY', 'TARGET_DAY_NEW', 'TARGET_DAY_EXIST']

# P : Nationwide
agg_p_df = merge_hh_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'FREQUENCY']).agg({'TARGET_MTH': 'sum', 'TARGET_MTH_NEW': 'sum', 'TARGET_MTH_EXIST': 'sum', 'TARGET_DAY': 'sum', 'TARGET_DAY_NEW': 'sum', 'TARGET_DAY_EXIST': 'sum'}).reset_index()
agg_p_df['AREA_NO'] = 1
agg_p_df['AREA_TYPE'] = 'P'
agg_p_df['AREA_CD'] = 'P'
agg_p_df['AREA_NAME'] = 'Nationwide'
agg_p_df = agg_p_df.loc[:, agg_cols]
# agg_p_df[agg_p_df['TM_KEY_DAY']==20240501]

# G : Region
agg_g_df = merge_hh_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'FREQUENCY', 'ORGID_G', 'TDS_SGMD']).agg({'TARGET_MTH': 'sum', 'TARGET_MTH_NEW': 'sum', 'TARGET_MTH_EXIST': 'sum', 'TARGET_DAY': 'sum', 'TARGET_DAY_NEW': 'sum', 'TARGET_DAY_EXIST': 'sum'}).reset_index()
agg_g_df['AREA_NO'] = 2
agg_g_df['AREA_TYPE'] = 'G'
agg_g_df.rename(columns={'ORGID_G': 'AREA_CD'}, inplace=True)
agg_g_df.rename(columns={'TDS_SGMD': 'AREA_NAME'}, inplace=True)
agg_g_df = agg_g_df.loc[:, agg_cols]
# agg_g_df[agg_g_df['TM_KEY_DAY']==20240501]

# H : HOP_HINT
agg_h_df = merge_hh_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'FREQUENCY', 'ORGID_H', 'HOP_HINT']).agg({'TARGET_MTH': 'sum', 'TARGET_MTH_NEW': 'sum', 'TARGET_MTH_EXIST': 'sum', 'TARGET_DAY': 'sum', 'TARGET_DAY_NEW': 'sum', 'TARGET_DAY_EXIST': 'sum'}).reset_index()
agg_h_df['AREA_NO'] = 3
agg_h_df['AREA_TYPE'] = 'H'
agg_h_df.rename(columns={'ORGID_H': 'AREA_CD'}, inplace=True)
agg_h_df.rename(columns={'HOP_HINT': 'AREA_NAME'}, inplace=True)
agg_h_df = agg_h_df.loc[:, agg_cols]
# agg_h_df[agg_h_df['TM_KEY_DAY']==20240501]

# HH : D_CLUSTER
agg_hh_df = merge_hh_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'FREQUENCY', 'ORGID_HH', 'D_CLUSTER']).agg({'TARGET_MTH': 'sum', 'TARGET_MTH_NEW': 'sum', 'TARGET_MTH_EXIST': 'sum', 'TARGET_DAY': 'sum', 'TARGET_DAY_NEW': 'sum', 'TARGET_DAY_EXIST': 'sum'}).reset_index()
agg_hh_df['AREA_NO'] = 4
agg_hh_df['AREA_TYPE'] = 'HH'
agg_hh_df['ORGID_HH'] = agg_hh_df['ORGID_HH'].astype(int).astype(str)
agg_hh_df.rename(columns={'ORGID_HH': 'AREA_CD'}, inplace=True)
agg_hh_df.rename(columns={'D_CLUSTER': 'AREA_NAME'}, inplace=True)
agg_hh_df = agg_hh_df.loc[:, agg_cols]
# agg_hh_df[agg_hh_df['TM_KEY_DAY']==20240601]

# Concat DataFrame
agg_all_area_df = pd.concat([agg_p_df, agg_g_df, agg_h_df, agg_hh_df], ignore_index=True)
# agg_all_area_df['AGG_TYPE'] = 'S'
# agg_all_area_df['FREQUENCY'] = np.where(agg_all_area_df['FREQUENCY']=='DAY', 'Daily', 'Bill Cycle')
# agg_all_area_df['REMARK'] = 'Allocate from 96 Cluster (HH level)'
# # agg_all_area_df['REMARK'] = agg_all_area_df['TM_KEY_MTH'].apply(lambda x: 'H Level 64 Province' if x>=202401 and x<=202403 else 'HH Level 96 Cluster')
# agg_all_area_df = agg_all_area_df[['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'TARGET_DAY', 'TARGET_DAY_NEW', 'TARGET_DAY_EXIST', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST', 'AGG_TYPE', 'FREQUENCY', 'REMARK']]

print(f'agg_all_area_df : {agg_all_area_df.shape[0]} rows, {agg_all_area_df.shape[1]} columns')
# agg_all_area_df.loc[agg_all_area_df['TM_KEY_DAY']==20240601]
agg_all_area_df.tail(3)

agg_all_area_df : 20469 rows, 19 columns


,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,FREQUENCY,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST,TARGET_DAY,TARGET_DAY_NEW,TARGET_DAY_EXIST
20466,2025,202506,2025027,20250630,TB4R000100,TVS Revenue,TRUE,T,4,HH,910096,SURIN,Bill Cycle,1.542973e+06,7120.439334,1.535852e+06,507127.176470,2340.267014,504786.909456
20467,2025,202506,2025027,20250630,TB4R000100,TVS Revenue,TRUE,T,4,HH,910097,UBON RATCHATHANI,Bill Cycle,2.443539e+06,11276.329565,2.432262e+06,803115.215958,3706.178914,799409.037044
20468,2025,202506,2025027,20250630,TB4R000100,TVS Revenue,TRUE,T,4,HH,910098,YASOTHON,Bill Cycle,5.491177e+05,2534.042912,5.465837e+05,180477.912475,832.861114,179645.051361


In [53]:
''' Aggregate TB2R000100 : Postpaid Revenue : TMH (Nationwide Only)'''

post_revenue_tmh_df = agg_all_area_df.loc[(agg_all_area_df['METRIC_CD'].isin(['TB2R010100', 'TB2R020100'])) & (agg_all_area_df['AREA_CD']=='P')]
post_revenue_tmh_df = post_revenue_tmh_df\
    .groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'FREQUENCY'])\
        .agg({'TARGET_MTH': 'sum', 'TARGET_MTH_NEW': 'sum', 'TARGET_MTH_EXIST': 'sum', 'TARGET_DAY': 'sum', 'TARGET_DAY_NEW': 'sum', 'TARGET_DAY_EXIST': 'sum'}).reset_index()
post_revenue_tmh_df['METRIC_CD'] = 'TB2R000100'
post_revenue_tmh_df['METRIC_NAME'] = 'Postpaid Revenue : TMH'
post_revenue_tmh_df = post_revenue_tmh_df[['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'FREQUENCY', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST', 'TARGET_DAY', 'TARGET_DAY_NEW', 'TARGET_DAY_EXIST']]

''' Test '''
# post_revenue_tmh_df = post_revenue_tmh_df.loc[post_revenue_tmh_df['PRODUCT_GRP']=='TOL']
# post_revenue_tmh_df = post_revenue_tmh_df.loc[post_revenue_tmh_df['AREA_KEY'].isin(['P', '902033'])]
# post_revenue_tmh_df = post_revenue_tmh_df.loc[post_revenue_tmh_df['TM_KEY_MTH']==202503]
# post_revenue_tmh_df = post_revenue_tmh_df.loc[post_revenue_tmh_df['TM_KEY_DAY']==20250302]
# post_revenue_tmh_df = post_revenue_tmh_df.loc[post_revenue_tmh_df['AREA_TYPE']=='P']

print(f'post_revenue_tmh_df : {post_revenue_tmh_df.shape[0]} rows, {post_revenue_tmh_df.shape[1]} columns')
post_revenue_tmh_df.tail(3)

post_revenue_tmh_df : 20 rows, 19 columns


,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,FREQUENCY,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST,TARGET_DAY,TARGET_DAY_NEW,TARGET_DAY_EXIST
17,2025,202506,2025026,20250625,TB2R000100,Postpaid Revenue : TMH,TRUE,T,1,P,P,Nationwide,Bill Cycle,3.691470e+09,3.428173e+08,3.343952e+09,3.692244e+08,3.428892e+07,3.344653e+08
18,2025,202506,2025026,20250628,TB2R000100,Postpaid Revenue : TMH,TRUE,T,1,P,P,Nationwide,Bill Cycle,3.691470e+09,3.428173e+08,3.343952e+09,3.973047e+08,3.689666e+07,3.599021e+08
19,2025,202506,2025027,20250630,TB2R000100,Postpaid Revenue : TMH,TRUE,T,1,P,P,Nationwide,Bill Cycle,3.691470e+09,3.428173e+08,3.343952e+09,2.588628e+07,2.403992e+06,2.344933e+07


In [54]:
''' Prepairing Latest Results '''

latest_concat_df = pd.concat([agg_all_area_df, post_revenue_tmh_df], ignore_index=True)
latest_concat_df.rename(columns={'TARGET_DAY': 'DAY_VALUE', 'TARGET_MTH': 'MTH_VALUE'}, inplace=True)
latest_concat_df['AGG_TYPE'] = 'S'
latest_concat_df['REMARK'] = 'Allocate from 96 Cluster (HH level)'

latest_results_df = latest_concat_df[['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'DAY_VALUE', 'MTH_VALUE', 'AGG_TYPE', 'FREQUENCY', 'REMARK']]
latest_results_df.tail(3)

,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,DAY_VALUE,MTH_VALUE,AGG_TYPE,FREQUENCY,REMARK
20486,2025,202506,2025026,20250625,TB2R000100,Postpaid Revenue : TMH,TRUE,T,1,P,P,Nationwide,3.692244e+08,3.691470e+09,S,Bill Cycle,Allocate from 96 Cluster (HH level)
20487,2025,202506,2025026,20250628,TB2R000100,Postpaid Revenue : TMH,TRUE,T,1,P,P,Nationwide,3.973047e+08,3.691470e+09,S,Bill Cycle,Allocate from 96 Cluster (HH level)
20488,2025,202506,2025027,20250630,TB2R000100,Postpaid Revenue : TMH,TRUE,T,1,P,P,Nationwide,2.588628e+07,3.691470e+09,S,Bill Cycle,Allocate from 96 Cluster (HH level)


In [55]:
''' Sample Data '''

chk_df = latest_results_df.loc[latest_results_df['TM_KEY_DAY']==20250502]

chk_df = chk_df\
    .groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'FREQUENCY', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE'])\
        .agg({'MTH_VALUE': 'sum'}).reset_index()

mod_col_list = chk_df.iloc[:, 8:].columns.tolist()
for col in mod_col_list:
    chk_df[col] = chk_df[col].apply(lambda x: format(x, ',.0f'))

chk_df

,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,FREQUENCY,METRIC_CD,METRIC_NAME,AREA_TYPE,MTH_VALUE
0,2025,202505,2025018,20250502,Bill Cycle,TB2R000100,Postpaid Revenue : TMH,P,"3,691,470,486"
1,2025,202505,2025018,20250502,Bill Cycle,TB2R010100,Postpaid Revenue B2C : TMH,G,"3,373,190,486"
2,2025,202505,2025018,20250502,Bill Cycle,TB2R010100,Postpaid Revenue B2C : TMH,H,"3,373,190,486"
3,2025,202505,2025018,20250502,Bill Cycle,TB2R010100,Postpaid Revenue B2C : TMH,HH,"3,373,190,486"
4,2025,202505,2025018,20250502,Bill Cycle,TB2R010100,Postpaid Revenue B2C : TMH,P,"3,373,190,486"
5,2025,202505,2025018,20250502,Bill Cycle,TB2R020100,Postpaid Revenue B2B : TMH,P,"318,280,000"
6,2025,202505,2025018,20250502,Bill Cycle,TB3R000100,TOL Revenue,G,"1,692,989,226"
7,2025,202505,2025018,20250502,Bill Cycle,TB3R000100,TOL Revenue,H,"1,692,989,226"
8,2025,202505,2025018,20250502,Bill Cycle,TB3R000100,TOL Revenue,HH,"1,692,989,226"
9,2025,202505,2025018,20250502,Bill Cycle,TB3R000100,TOL Revenue,P,"1,692,989,226"


In [ ]:
# main_kpi_df = merge_hh_df
# main_kpi_df = main_kpi_df[['CUST_TYPE', 'TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH', 'PERIODFLAG', 'PRODUCT_GRP', 'FREQUENCY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_KEY', 'AREA_DESC', 'TARGET_MTH', 'TARGET_DAY']]

# main_kpi_df.tail(3)

### Step 3 : Insert to "INTERIM_VINSIGHT_DATA"
    Delete -> Insert

In [56]:
''' Input Parameter '''

# Create list
month_list = latest_results_df['TM_KEY_MTH'].drop_duplicates().tolist()
mt_cd_list = latest_results_df['METRIC_CD'].drop_duplicates().tolist()

if len(mt_cd_list) == 1:
    mt_cd_list = str(mt_cd_list).replace(r'[', '(').replace(r']', ')')
else:
    mt_cd_list = tuple(mt_cd_list)

# Create Param
# v_param = dict(mth_start=202406, mth_end=202408, metric_cd=mt_cd_list)
v_param = dict(mth_start=min(month_list), mth_end=max(month_list), metric_cd=mt_cd_list)
v_target_schema = 'AUTOKPI'
v_target_table = 'INTERIM_VINSIGHT_DATA'

# query_delete = f"DELETE {v_target_schema}.{v_target_table} WHERE TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']} AND METRIC_CD IN {v_param['metric_cd']}"
query_delete = f"""
    DELETE {v_target_schema}.{v_target_table} 
    WHERE VERSION = 'T'
    AND METRIC_CD IN {v_param['metric_cd']}
    AND TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']} 
"""

print(f"\nParameter...\n\n   -> TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']}\n   -> METRIC_CD IN {v_param['metric_cd']}")
print(f'\nDataFrame...\n\n   -> latest_results_df : {latest_results_df.shape[0]} rows, {latest_results_df.shape[1]} columns') 
print(f'\nquery_delete...\n{query_delete}')


Parameter...

   -> TM_KEY_MTH BETWEEN 202505 AND 202506
   -> METRIC_CD IN ('TB1R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100', 'TB2R000100')

DataFrame...

   -> latest_results_df : 20489 rows, 17 columns

query_delete...

    DELETE AUTOKPI.INTERIM_VINSIGHT_DATA 
    WHERE VERSION = 'T'
    AND METRIC_CD IN ('TB1R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100', 'TB2R000100')
    AND TM_KEY_MTH BETWEEN 202505 AND 202506 



In [57]:
''' DELETE -> INSERT '''

job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')


# Create rows from DataFrame
rows = [tuple(x) for x in latest_results_df.values]


# Connect : AKPIPRD
dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
conn = oracledb.connect(dsn)
print(f'\n{AKPIPRD_db} : Connected')
cur = conn.cursor()
print(f'\nProcessing...')


try:
    # # Truncate
    # cur.execute(f"TRUNCATE TABLE {v_target_schema}.{v_target_table}")
    # print(f'\n   -> TRUNCATE : "{v_target_table}" : Done !')

    # Delete
    cur.execute(query_delete)
    print(f'\n   -> DELETE : "{v_target_table}" : Done !')
    
    # Insert
    cur.executemany(f"""
        INSERT INTO {v_target_table} 
        (TM_KEY_YR, TM_KEY_MTH, TRUE_TM_KEY_WK, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, VERSION, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, DAY_VALUE, MTH_VALUE, AGG_TYPE, FREQUENCY, REMARK) 
        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17)
        """, rows)
    print(f'\n   -> INSERT : "{v_target_table}" : Done !')

    cur.close()
    conn.commit()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    print(f'\nJob Done !!!')



Job Start... 2025-05-20, 15:05:17

AKPIPRD : Connected

Processing...

   -> DELETE : "INTERIM_VINSIGHT_DATA" : Done !

   -> INSERT : "INTERIM_VINSIGHT_DATA" : Done !

AKPIPRD : Disconnected

Job Done !!!


### Step 4 : Check Result "INTERIM_VINSIGHT_DATA"

In [2]:
''' Create Result DataFrame '''

# Connect : AKPIPRD
tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
tgt_conn = oracledb.connect(tgt_dsn)
tgt_cur = tgt_conn.cursor()


try:
    # Get : Result Data Summary
    tgt_cur.execute("""
        SELECT TM_KEY_MTH, METRIC_CD, METRIC_NAME, COMP_CD, VERSION
            , SUM(CASE WHEN AREA_TYPE = 'P' THEN DAY_VALUE END) AS P
            , SUM(CASE WHEN AREA_TYPE = 'G' THEN DAY_VALUE END) AS G
            , SUM(CASE WHEN AREA_TYPE = 'H' THEN DAY_VALUE END) AS H
            , SUM(CASE WHEN AREA_TYPE = 'HH' THEN DAY_VALUE END) AS HH
            , MAX(LOAD_DATE) LOAD_DATE
        FROM AUTOKPI.INTERIM_VINSIGHT_DATA
        WHERE VERSION = 'T'
        AND METRIC_CD IN (
            'TB1R000100' --Prepaid Revenue : TMH
            , 'TB1R000101' --Prepaid New Revenue : TMH (2024 only)
            , 'TB1R000102' --Prepaid Existing Revenue : TMH (2024 only)
            , 'TB2R000100' --Postpaid Revenue : TMH
            , 'TB2R000101' --Postpaid New Revenue : TMH (2024 only)
            , 'TB2R000102' --Postpaid Existing Revenue : TMH (2024 only)
            , 'TB2R010100' --Postpaid Revenue B2C : TMH
            , 'TB2R020100' --Postpaid Revenue B2B : TMH
            , 'TB3R000100' --TOL Revenue
            , 'TB3R000101' --TOL New Revenue (2024 only)
            , 'TB3R000102' --TOL Existing Revenue (2024 only)
            , 'TB4R000100' --TVS Revenue
            , 'TB4R000101' --TVS New Revenue (2024 only)
            , 'TB4R000102' --TVS Existing Revenue (2024 only)
            )
        AND TM_KEY_MTH >= 202501
        GROUP BY TM_KEY_MTH, METRIC_CD, METRIC_NAME, COMP_CD, VERSION
        --ORDER BY TM_KEY_MTH, METRIC_CD
    """)
    rows = tgt_cur.fetchall()
    print(f'\nGet : Fact Summary...')
    chk_result_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in tgt_cur.description])
    print(f'\n   -> chk_result_df : {chk_result_df.shape[0]} rows, {chk_result_df.shape[1]} columns') 
    
    # # Display
    # tmp_result_df = chk_result_df.copy()
    # # tmp_result_df = tmp_result_df.replace(np.nan, None)
    # # tmp_result_df.iloc[:, 4:18] = tmp_result_df.iloc[:, 4:18].fillna(0)
    # mod_col_list = tmp_result_df.iloc[:, 5:9].columns.tolist()
    # for col in mod_col_list:
    #     tmp_result_df[col] = tmp_result_df[col].apply(lambda x: format(x, ',.2f') if re.search('%', col) else format(x, ',.0f'))
    # print(f'\n{tmp_result_df.to_string(max_cols=10)}') #max_rows=1000

    tgt_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    tgt_conn.close()


Get : Fact Summary...

   -> chk_result_df : 36 rows, 10 columns


In [3]:
m4_df = chk_result_df.loc[chk_result_df['TM_KEY_MTH']==202504].copy()
# m4_df = m4_df.replace(np.nan, None)
m4_df.iloc[:, 5:9] = m4_df.iloc[:, 5:9].fillna(0)
mod_col_list = m4_df.iloc[:, 5:9].columns.tolist()
for col in mod_col_list:
    m4_df[col] = m4_df[col].apply(lambda x: format(x, ',.2f') if re.search('%', col) else format(x, ',.0f'))
m4_df = m4_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
m4_df

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,P,G,H,HH,LOAD_DATE
0,202504,TB1R000100,Prepaid Revenue : TMH,TRUE,T,"2,488,191,889","2,488,191,889","2,488,191,889","2,488,191,889",2025-04-28 15:46:42.603195
1,202504,TB2R000100,Postpaid Revenue : TMH,TRUE,T,"3,676,555,973",0,0,0,2025-04-28 15:46:42.603195
2,202504,TB2R010100,Postpaid Revenue B2C : TMH,TRUE,T,"3,358,275,973","3,358,275,973","3,358,275,973","3,358,275,973",2025-04-28 15:46:42.603195
3,202504,TB2R020100,Postpaid Revenue B2B : TMH,TRUE,T,"318,280,000",0,0,0,2025-04-28 15:46:42.603195
4,202504,TB3R000100,TOL Revenue,TRUE,T,"1,672,918,006","1,672,918,006","1,672,918,006","1,672,918,006",2025-04-28 15:46:42.603195
5,202504,TB4R000100,TVS Revenue,TRUE,T,"379,499,818","379,499,818","379,499,818","379,499,818",2025-04-28 15:46:42.603195


In [4]:
m5_df = chk_result_df.loc[chk_result_df['TM_KEY_MTH']==202505].copy()
m5_df.iloc[:, 5:9] = m5_df.iloc[:, 5:9].fillna(0)
mod_col_list = m5_df.iloc[:, 5:9].columns.tolist()
for col in mod_col_list:
    m5_df[col] = m5_df[col].apply(lambda x: format(x, ',.2f') if re.search('%', col) else format(x, ',.0f'))
m5_df = m5_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
m5_df

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,P,G,H,HH,LOAD_DATE
0,202505,TB1R000100,Prepaid Revenue : TMH,TRUE,T,"2,611,860,019","2,611,860,019","2,611,860,019","2,611,860,019",2025-05-20 15:05:56.236203
1,202505,TB2R000100,Postpaid Revenue : TMH,TRUE,T,"3,691,470,486",0,0,0,2025-05-20 15:05:56.236203
2,202505,TB2R010100,Postpaid Revenue B2C : TMH,TRUE,T,"3,373,190,486","3,373,190,486","3,373,190,486","3,373,190,486",2025-05-20 15:05:56.236203
3,202505,TB2R020100,Postpaid Revenue B2B : TMH,TRUE,T,"318,280,000",0,0,0,2025-05-20 15:05:56.236203
4,202505,TB3R000100,TOL Revenue,TRUE,T,"1,692,989,226","1,692,989,226","1,692,989,226","1,692,989,226",2025-05-20 15:05:56.236203
5,202505,TB4R000100,TVS Revenue,TRUE,T,"377,894,818","377,894,818","377,894,818","377,894,818",2025-05-20 15:05:56.236203


In [5]:
m6_df = chk_result_df.loc[chk_result_df['TM_KEY_MTH']==202506].copy()
m6_df.iloc[:, 5:9] = m6_df.iloc[:, 5:9].fillna(0)
mod_col_list = m6_df.iloc[:, 5:9].columns.tolist()
for col in mod_col_list:
    m6_df[col] = m6_df[col].apply(lambda x: format(x, ',.2f') if re.search('%', col) else format(x, ',.0f'))
m6_df = m6_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
m6_df

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,P,G,H,HH,LOAD_DATE
0,202506,TB1R000100,Prepaid Revenue : TMH,TRUE,T,"2,611,860,019","2,611,860,019","2,611,860,019","2,611,860,019",2025-05-20 15:05:56.236203
1,202506,TB2R000100,Postpaid Revenue : TMH,TRUE,T,"3,691,470,486",0,0,0,2025-05-20 15:05:56.236203
2,202506,TB2R010100,Postpaid Revenue B2C : TMH,TRUE,T,"3,373,190,486","3,373,190,486","3,373,190,486","3,373,190,486",2025-05-20 15:05:56.236203
3,202506,TB2R020100,Postpaid Revenue B2B : TMH,TRUE,T,"318,280,000",0,0,0,2025-05-20 15:05:56.236203
4,202506,TB3R000100,TOL Revenue,TRUE,T,"1,692,989,226","1,692,989,226","1,692,989,226","1,692,989,226",2025-05-20 15:05:56.236203
5,202506,TB4R000100,TVS Revenue,TRUE,T,"377,894,818","377,894,818","377,894,818","377,894,818",2025-05-20 15:05:56.236203


In [6]:
m7_df = chk_result_df.loc[chk_result_df['TM_KEY_MTH']==202507].copy()
m7_df.iloc[:, 5:9] = m7_df.iloc[:, 5:9].fillna(0)
mod_col_list = m7_df.iloc[:, 5:9].columns.tolist()
for col in mod_col_list:
    m7_df[col] = m7_df[col].apply(lambda x: format(x, ',.2f') if re.search('%', col) else format(x, ',.0f'))
m7_df = m7_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
m7_df

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,P,G,H,HH,LOAD_DATE
